In [1]:
#connecting colab with google drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#Path variables where training and testing datasets are

train_dir = "/content/drive/MyDrive/tomato/train"
val_dir   = "/content/drive/MyDrive/tomato/val"

In [3]:
#listing training directories

import os
os.listdir(train_dir)

['Tomato___Septoria_leaf_spot',
 'Tomato___Late_blight',
 'Tomato___healthy',
 'Tomato___Early_blight']

In [4]:
#creating directories

base_project = "/content/drive/MyDrive/plant_project"
os.makedirs(base_project + "/models", exist_ok=True)
os.makedirs(base_project + "/notebooks", exist_ok=True)
os.makedirs(base_project + "/results", exist_ok=True)


In [5]:
#importing libraries

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Model

In [6]:
img_size = 224
batch_size = 32

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.3,
    horizontal_flip=True,
    vertical_flip=True,  # Leaves can be oriented any way
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)

train_gen = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='categorical'
)

val_gen = val_datagen.flow_from_directory(
    val_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='categorical'
)

Found 4000 images belonging to 4 classes.
Found 400 images belonging to 4 classes.


In [7]:
base_model = MobileNetV2(
    weights='imagenet',
    include_top=False,
    input_shape=(224, 224, 3)
)

base_model.trainable = True

# Freeze early layers, fine-tune later layers
for layer in base_model.layers[:100]:
    layer.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(4, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output)


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [8]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-5),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [9]:
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=20
)

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 1200s 9s/step - accuracy: 0.3334 - loss: 1.5575 - val_accuracy: 0.5100 - val_loss: 1.2258
Epoch 2/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 61s 488ms/step - accuracy: 0.5722 - loss: 1.0252 - val_accuracy: 0.6275 - val_loss: 1.0348
Epoch 3/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 61s 483ms/step - accuracy: 0.6566 - loss: 0.8373 - val_accuracy: 0.6625 - val_loss: 0.9398
Epoch 4/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 62s 498ms/step - accuracy: 0.7247 - loss: 0.6897 - val_accuracy: 0.6900 - val_loss: 0.8769
Epoch 5/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 62s 492ms/step - accuracy: 0.7577 - loss: 0.6060 - val_accuracy: 0.6975 - val_loss: 0.8759
Epoch 6/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 62s 493ms/step - accuracy: 0.7938 - loss: 0.5168 - val_accuracy: 0.7200 - val_loss: 0.8132
Epoch 7/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 62s 495ms/step - accuracy: 0.7933 - loss: 0.5174 - val_accuracy: 0.7325 - val_loss: 0.7739
Epoch 8/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 61s 483ms/step - accuracy: 0.8263 - loss: 0.

In [10]:
model.save("/content/drive/MyDrive/plant_project/models/plant_disease_model.keras")

In [11]:
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np

val_gen.reset()
pred = model.predict(val_gen)
y_pred = np.argmax(pred, axis=1)

cm = confusion_matrix(val_gen.classes, y_pred)
print(cm)

print(classification_report(val_gen.classes, y_pred))


13/13 ━━━━━━━━━━━━━━━━━━━━ 11s 461ms/step
[[17 30 37 16]
 [18 22 31 29]
 [17 32 26 25]
 [14 32 26 28]]
              precision    recall  f1-score   support

           0       0.26      0.17      0.20       100
           1       0.19      0.22      0.20       100
           2       0.22      0.26      0.24       100
           3       0.29      0.28      0.28       100

    accuracy                           0.23       400
   macro avg       0.24      0.23      0.23       400
weighted avg       0.24      0.23      0.23       400

